In [82]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import PillowWriter, FuncAnimation

from src.io.datasets import CSVReader
from src.config.settings import get_settings

%matplotlib widget

In [83]:
settings = get_settings()

reader = CSVReader()

In [84]:
prob_blitz = reader.read(settings.FRAME_DISGUISE_RESULTS_FILE)

In [85]:
prob_blitz = prob_blitz[(prob_blitz.gameId == 2022101300) & (prob_blitz.playId == 291)].sort_values(by="frameId")
prob_blitz['time'] = prob_blitz['time'].apply(lambda x: pd.to_datetime(x) if '.' in x else pd.to_datetime(x + '.000'))

kyler_gordon_blitz_prob = prob_blitz[(prob_blitz.jerseyNumber == 6)]
jaquan_brisker_blitz_prob = prob_blitz[(prob_blitz.jerseyNumber == 9)]

In [86]:
kyler_gordon_blitz_prob

,gameId,playId,nflId,jerseyNumber,club,position,displayName,frameId,time,wasInitialPassRusher,blitz_probability,frame_disguise,num_frames
102716,2022101300,291,54504,6,CHI,2,Kyler Gordon,69,2022-10-14 00:25:16.800,1,0.001588,0.998412,85
102717,2022101300,291,54504,6,CHI,2,Kyler Gordon,70,2022-10-14 00:25:16.900,1,0.001455,0.998545,85
102718,2022101300,291,54504,6,CHI,2,Kyler Gordon,71,2022-10-14 00:25:17.000,1,0.001516,0.998484,85
102719,2022101300,291,54504,6,CHI,2,Kyler Gordon,72,2022-10-14 00:25:17.100,1,0.001635,0.998365,85
102720,2022101300,291,54504,6,CHI,2,Kyler Gordon,73,2022-10-14 00:25:17.200,1,0.000857,0.999143,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102796,2022101300,291,54504,6,CHI,2,Kyler Gordon,149,2022-10-14 00:25:24.800,1,0.030050,0.969950,85
102797,2022101300,291,54504,6,CHI,2,Kyler Gordon,150,2022-10-14 00:25:24.900,1,0.040581,0.959419,85
102798,2022101300,291,54504,6,CHI,2,Kyler Gordon,151,2022-10-14 00:25:25.000,1,0.040581,0.959419,85
102799,2022101300,291,54504,6,CHI,2,Kyler Gordon,152,2022-10-14 00:25:25.100,1,0.029355,0.970645,85


In [87]:
kyler_gordon_blitz_prob = kyler_gordon_blitz_prob.copy()
jaquan_brisker_blitz_prob = jaquan_brisker_blitz_prob.copy()

# Calculate the time in seconds
kyler_gordon_blitz_prob['time_seconds'] = (
    kyler_gordon_blitz_prob['time'] - kyler_gordon_blitz_prob['time'].iloc[0]
).dt.total_seconds()
time = kyler_gordon_blitz_prob['time_seconds'].values
kyler_gordon = kyler_gordon_blitz_prob['blitz_probability'].values
jaquan_brisker = jaquan_brisker_blitz_prob['blitz_probability'].values

# Colors for the lines
kyler_color = "#2a9d8f"
jaquan_color = "#e76f51"

# Create the figure and axes
fig, ax = plt.subplots()

# Initialize the lines
kyler_gordon_line, = ax.plot([], [], label="Kyler Gordon (#6)", lw=2, color=kyler_color)
jaquan_brisker_line, = ax.plot([], [], label="Jaquan Brisker (#9)", lw=2, color=jaquan_color)

# Configure the axes
ax.set_xlabel('Time after Lineset (s)', fontsize=12)
ax.set_ylabel('Blitz Probability', fontsize=12)
ax.set_xlim([0, max(time)])
ax.set_ylim([0, 1])
ax.legend(loc="lower left", bbox_to_anchor=(0, 0.1))

# Update function for the animation
def update(frame):
    kyler_gordon_line.set_data(time[:frame], kyler_gordon[:frame])
    jaquan_brisker_line.set_data(time[:frame], jaquan_brisker[:frame])
    return kyler_gordon_line, jaquan_brisker_line

# Create the animation
fig.tight_layout()
frames_with_pause = list(range(len(time))) + [len(time)] * 33  # Add 50 frames of pause = 5s
ani = FuncAnimation(fig=fig, func=update, frames=frames_with_pause, interval=100)

# Save the animation as a GIF
ani.save("test.gif", writer=PillowWriter(fps=10, bitrate=1800))

plt.close(fig)